In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def start_x_driver():
    options = Options()
    # X için giriş yapman gerekeceği için tarayıcıyı görünür (headful) açıyoruz
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("detach", True)
    
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

print("✅ Kütüphaneler hazır.")

✅ Kütüphaneler hazır.


In [2]:
driver = start_x_driver()
# Doğrudan Arçelik Destek yanıtlar sekmesine gidiyoruz
driver.get("https://x.com/ArcelikDestek/with_replies")

print("⚠️ LÜTFEN DİKKAT: Açılan pencerede X hesabına giriş yap.")
print("Giriş yaptıktan sonra sayfanın tamamen yüklenmesini bekle ve bir sonraki hücreye geç.")

⚠️ LÜTFEN DİKKAT: Açılan pencerede X hesabına giriş yap.
Giriş yaptıktan sonra sayfanın tamamen yüklenmesini bekle ve bir sonraki hücreye geç.


In [4]:
from IPython.display import display

def x_safe_scraper(target=5):
    complaints = []
    seen_texts = set()
    
    # Engellenecek kurumsal anahtar kelimeler
    corporate_names = ["arcelik", "arçelik", "destek", "müşteri hizmetleri"]
    
    print(f"🕵️ Filtreli tarama başladı... Hedef: {target} gerçek şikayet.")
    print("-" * 50)

    while len(complaints) < target:
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        
        for tweet in tweets:
            try:
                # Kullanıcı bilgisini ve tweet metnini al
                user_element = tweet.find_element(By.CSS_SELECTOR, '[data-testid="User-Name"]')
                text_element = tweet.find_element(By.CSS_SELECTOR, '[data-testid="tweetText"]')
                
                u_info = user_element.text.lower() # Küçük harfe çevirerek kontrol et
                t_text = text_element.text
                
                # SIKI FİLTRELEME:
                # 1. Kullanıcı adında veya görünen isimde 'arcelik', 'destek' vb. geçmemeli
                is_corporate = any(name in u_info for name in corporate_names)
                
                # 2. Daha önce bu metni almamış olmalıyız
                if not is_corporate and t_text not in seen_texts:
                    complaints.append({
                        "Yazar": user_element.text.split('\n')[0],
                        "Kullanıcı Adı": user_element.text.split('\n')[1] if '\n' in user_element.text else "",
                        "Şikayet Mesajı": t_text
                    })
                    seen_texts.add(t_text)
                    
                    print(f"✅ {len(complaints)}. GERÇEK ŞİKAYET YAKALANDI")
                    print(f"👤 Yazar: {complaints[-1]['Yazar']}")
                    print(f"💬 Mesaj: {t_text[:100]}...")
                    print("-" * 30)
                
                if len(complaints) >= target: break
            except:
                continue
        
        if len(complaints) < target:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(4)
            
    return complaints

# Testi 5 adet gerçek kullanıcı için tekrar başlat
results = x_safe_scraper(5)

if results:
    print("\n🎯 FİLTRELEME BAŞARILI! Gerçek kullanıcı tablosu:")
    display(pd.DataFrame(results))

🕵️ Filtreli tarama başladı... Hedef: 5 gerçek şikayet.
--------------------------------------------------
✅ 1. GERÇEK ŞİKAYET YAKALANDI
👤 Yazar: Emine
💬 Mesaj: @ArcelikDestek
 ay geçti bulaşık makinasını tamir edemeyen Edirne yetkili servisiniz parça değişim s...
------------------------------
✅ 2. GERÇEK ŞİKAYET YAKALANDI
👤 Yazar: Emine
💬 Mesaj: Ayrıca bugün yeni ürünü teslim aldım ve kullanılır şekilde montajı yapıldı,garanti süreniz esasen bu...
------------------------------
✅ 3. GERÇEK ŞİKAYET YAKALANDI
👤 Yazar: Emine
💬 Mesaj: Nihayetinde defolu ürün değiştirildi, ancak montaj yapılmadığı halde defolu ürünün bana teslim edild...
------------------------------
✅ 4. GERÇEK ŞİKAYET YAKALANDI
👤 Yazar: Oğuz Alper Öktem
💬 Mesaj: İstanbul’daki yerel yönetimlerin bilmesi için yazıyorum:

İstanbul’da:
318.247 TAG sürücüsü,
6.883.8...
------------------------------
✅ 5. GERÇEK ŞİKAYET YAKALANDI
👤 Yazar: ersin cengiz @devrimdeniz
💬 Mesaj: @arcelik
 @kocholding
 @FOXhaber 
@fatihportakal
 @Ar

,Yazar,Kullanıcı Adı,Şikayet Mesajı
0,Emine,@emyyjusthuman,@ArcelikDestek\n ay geçti bulaşık makinasını t...
1,Emine,@emyyjusthuman,Ayrıca bugün yeni ürünü teslim aldım ve kullan...
2,Emine,@emyyjusthuman,"Nihayetinde defolu ürün değiştirildi, ancak mo..."
3,Oğuz Alper Öktem,@OguzAlperOktem,İstanbul’daki yerel yönetimlerin bilmesi için ...
4,ersin cengiz @devrimdeniz,@ersincengiz,@arcelik\n @kocholding\n @FOXhaber \n@fatihpor...


In [6]:
import pandas as pd
import time
import datetime

# Pandas görünüm ayarı (Hücrede tam metinleri görmek için)
pd.set_option('display.max_colwidth', None)

def x_mega_scraper(target_count=5000):
    complaints_list = []
    seen_texts = set()
    # Arçelik ve destek ekiplerini ayıklamak için kara liste
    corporate_blacklist = ["arcelik", "arçelik", "destek", "müşteri hizmetleri", "servis", "berke", "merve"]
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    no_new_tweet_count = 0
    
    print(f"🚀 Dev Operasyon Başladı: Hedef {target_count} Gerçek Şikayet")
    print("📢 İpucu: Eğer sayfa donarsa manuel olarak biraz aşağı kaydırın.")
    print("-" * 60)

    while len(complaints_list) < target_count:
        # Mevcut sayfadaki tüm tweetleri yakala
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        
        found_in_this_scroll = 0
        for tweet in tweets:
            try:
                # 1. Elementleri Yakala
                user_element = tweet.find_element(By.CSS_SELECTOR, '[data-testid="User-Name"]')
                text_element = tweet.find_element(By.CSS_SELECTOR, '[data-testid="tweetText"]')
                time_element = tweet.find_element(By.TAG_NAME, 'time')
                
                u_info = user_element.text
                t_text = text_element.text
                t_date = time_element.get_attribute('datetime')
                
                # 2. Sıkı Filtre Uygula
                is_corporate = any(name in u_info.lower() for name in corporate_blacklist)
                
                if not is_corporate and t_text not in seen_texts:
                    complaints_list.append({
                        "Veri Kaynağı": "X (Twitter)",
                        "Toplanma Tarihi": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                        "Tweet Tarihi": t_date,
                        "Yazar": u_info.split('\n')[0],
                        "Kullanıcı Adı": u_info.split('\n')[1] if '\n' in u_info else "",
                        "Şikayet Mesajı": t_text
                    })
                    seen_texts.add(t_text)
                    found_in_this_scroll += 1
                
                if len(complaints_list) >= target_count: break
            except:
                continue

        # 3. İlerleme Raporu ve Otomatik Yedekleme
        if found_in_this_scroll > 0:
            print(f"📊 Toplam: {len(complaints_list)} | Bu kaydırmada {found_in_this_scroll} yeni şikayet eklendi.")
            no_new_tweet_count = 0
        else:
            no_new_tweet_count += 1

        # Her 250 kayıtta bir Excel yedekle (Çökme riskine karşı)
        if len(complaints_list) > 0 and len(complaints_list) % 250 == 0:
            pd.DataFrame(complaints_list).to_excel(f"arcelik_X_YEDEK_{len(complaints_list)}.xlsx", index=False)
            print(f"💾 KRİTİK YEDEK: {len(complaints_list)} kayıt kaydedildi.")

        # 4. Kaydırma ve Akış Kontrolü
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4) # X'in rate limitine takılmamak için bekleme
        
        # Eğer sayfa sonuna gelindiyse veya X yüklemeyi durdurduysa
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height and no_new_tweet_count > 5:
            print("🛑 Sayfa akışı durdu. Manuel müdahale gerekebilir veya veriler bitti.")
            break
        last_height = new_height

    return pd.DataFrame(complaints_list)

# --- ÇALIŞTIR ---
# Not: 5.000 adet için bu işlem bir kaç saat sürebilir. 
# Arada bir tarayıcı penceresine bakıp X'in 'Giriş yap' uyarısı çıkarıp çıkarmadığını kontrol edin.
df_mega_x = x_mega_scraper(5000)

# Final Dosyası
df_mega_x.to_excel("arcelik_X_5000_Sikayet_Analiz_Hazir.xlsx", index=False)
print("\n🏆 OPERASYON TAMAMLANDI! 5.000 (veya ulaşılan maksimum) şikayet Excel'e döküldü.")

🚀 Dev Operasyon Başladı: Hedef 5000 Gerçek Şikayet
📢 İpucu: Eğer sayfa donarsa manuel olarak biraz aşağı kaydırın.
------------------------------------------------------------
📊 Toplam: 14 | Bu kaydırmada 14 yeni şikayet eklendi.
📊 Toplam: 26 | Bu kaydırmada 12 yeni şikayet eklendi.
📊 Toplam: 38 | Bu kaydırmada 12 yeni şikayet eklendi.
📊 Toplam: 51 | Bu kaydırmada 13 yeni şikayet eklendi.
📊 Toplam: 63 | Bu kaydırmada 12 yeni şikayet eklendi.
📊 Toplam: 74 | Bu kaydırmada 11 yeni şikayet eklendi.
📊 Toplam: 84 | Bu kaydırmada 10 yeni şikayet eklendi.
📊 Toplam: 95 | Bu kaydırmada 11 yeni şikayet eklendi.
📊 Toplam: 104 | Bu kaydırmada 9 yeni şikayet eklendi.
📊 Toplam: 118 | Bu kaydırmada 14 yeni şikayet eklendi.
📊 Toplam: 132 | Bu kaydırmada 14 yeni şikayet eklendi.
📊 Toplam: 146 | Bu kaydırmada 14 yeni şikayet eklendi.
📊 Toplam: 156 | Bu kaydırmada 10 yeni şikayet eklendi.
📊 Toplam: 168 | Bu kaydırmada 12 yeni şikayet eklendi.
📊 Toplam: 181 | Bu kaydırmada 13 yeni şikayet eklendi.
📊 Toplam